In [1]:
import sys
import pandas as pd 
sys.path.insert(0, '/home/bekah/gPhoton2')
from gPhoton.pipeline import execute_pipeline
from astropy.io import fits
from io import BytesIO
import numpy as np 
import os
import matplotlib.pyplot as plt
from gPhoton.coadd import zero_flag_and_edge
import pyarrow.parquet as parquet
from quickbin import bin2d

from photutils.background import Background2D, MedianBackground
from astropy.stats import SigmaClip

In [2]:
# make a list of fuv to make / look at 

og_aspect = parquet.read_table('/home/bekah/gPhoton2/gPhoton/aspect/metadata.parquet').to_pandas()
og_aspect = og_aspect.dropna(subset=['fuv_temp'])

In [3]:
og_aspect = og_aspect[og_aspect['legs'] == 0]

sampled_eclipses = og_aspect.sample(n=200, random_state=2)
sampled_eclipses

,eclipse,obstype,legs,ra_max,ra_min,dec_max,dec_min,fuv_temp
8261,13338,DIS,0,53.120694,53.091952,-29.389283,-29.414299,26.811481
971,2870,DIS,0,53.147050,53.119313,-27.857205,-27.880872,27.756960
5456,9405,GII,0,150.396398,150.373411,-6.591745,-6.615505,27.369032
13246,19188,GII,0,47.529317,47.491357,-53.324985,-53.347183,28.140380
3092,6307,NGS,0,273.377713,273.351977,9.065354,9.039640,29.535768
...,...,...,...,...,...,...,...,...
5634,9757,GII,0,122.818798,122.790868,25.218543,25.193259,31.382072
4301,7788,GII,0,2.483505,2.454958,25.940227,25.914979,29.628210
22304,30627,MIS,0,131.845922,131.817471,26.710279,26.684506,30.203828
18480,25782,MIS,0,125.298838,125.282610,8.016765,8.000432,29.571084


In [4]:
for index, eclipse in sampled_eclipses['eclipse'].items():
    print(f"running {eclipse}")
    try: 
        execute_pipeline(
                int(eclipse),
                "FUV",
                depth=None,
                # integer; None to deactivate (default None)
                threads=4,
                # where to both write output data and look for input data
                local_root="/media/bekah/BekahA/background_test/pics_and_photonlists",
                # auxiliary remote location for input data
                # remote_root="/mnt/s3",
                recreate=True,
                # list of floats; relevant only to lightcurve / photometry portion
                aperture_sizes=[12.8],
                # actually write image/movie products? otherwise hold in memory but
                # discard (possibly after performing photometry).
                write={"movie": False, "image": True},
                coregister_lightcurves=False,
                # photonpipe, moviemaker, None (default None)
                stop_after=None,
                photometry_only=False,
                # None, "gzip", "rice"
                compression="rice",
                # use array sparsification on movie frames?
                lil=False,
                # write movie frames as separate files
                burst=False,
                extended_photonlist=True,
                # aspect file, don't need to set unless need to use alt
                # file, 'aspect2.parquet'
            )
        
    except Exception as e: 
        print(e)


running 13338


/home/bekah/gPhoton2/gPhoton/pipeline.py:231: UserWarning: lil=False is deprected and will be removed in a future release. Defaulting to lil=True.
  warnings.warn("lil=False is deprected and will be removed in a future release. Defaulting to lil=True.")


starting timer
eclipse 13338 FUV  -- DIS; 0 leg(s)
downloading raw6file
Processing eclipse 13338
Offsetting FUV image for eclipse 13338 at 26.8114806821977 degrees.
Setting FUV offsets to x=-0.19512, y=1.0471003986134872
trange= ( 814498937.995 , 814499455.995 )                   
RA AVG: 53.10677801796305, DEC AVG: -29.401044353854793, ROLL AVG: 153.99720944406084
Loading raw6 file...                                        
829695 events
Unpacking raw6 data...                                      
stim_coef0, stim_coef1 = -442204.7380014617, 0.0005491652432270735
writing table to /media/bekah/BekahA/background_test/pics_and_photonlists/e13338/e13338-fd-b00.parquet
                                                            
Runtime statistics:
 runtime		=	2.94 sec. = (0.05 min.)
  processed	   =   827344 of 829695 events.
rate		=	282396.97 photons/sec.

6.95 elapsed seconds, restarting timer
making images from /media/bekah/BekahA/background_test/pics_and_photonlists/e13338/e13338-fd-b

/home/bekah/gPhoton2/gPhoton/pipeline.py:231: UserWarning: lil=False is deprected and will be removed in a future release. Defaulting to lil=True.
  warnings.warn("lil=False is deprected and will be removed in a future release. Defaulting to lil=True.")


Processing eclipse 2870
Offsetting FUV image for eclipse 2870 at 27.7569599930121 degrees.
Setting FUV offsets to x=-0.12628, y=0.6716214905135479
trange= ( 752553481.995 , 752553578.995 )                   
RA AVG: 53.13074583985039, DEC AVG: -27.866649846106775, ROLL AVG: 164.99730308245748
Loading raw6 file...                                        
197830 events
Unpacking raw6 data...                                      
stim_coef0, stim_coef1 = -582834.4245936471, 0.0007812404377208517
writing table to /media/bekah/BekahA/background_test/pics_and_photonlists/e02870/e02870-fd-b00.parquet
                                                            
Runtime statistics:
 runtime		=	1.15 sec. = (0.02 min.)
  processed	   =   193814 of 197830 events.
rate		=	172567.84 photons/sec.

3.54 elapsed seconds, restarting timer
making images from /media/bekah/BekahA/background_test/pics_and_photonlists/e02870/e02870-fd-b00.parquet
indexing data and making WCS solution
making full-depth image
0

...

/home/bekah/gPhoton2/gPhoton/pipeline.py:231: UserWarning: lil=False is deprected and will be removed in a future release. Defaulting to lil=True.
  warnings.warn("lil=False is deprected and will be removed in a future release. Defaulting to lil=True.")


Processing eclipse 22118
Offsetting FUV image for eclipse 22118 at 29.1303178649223 degrees.
Setting FUV offsets to x=-0.00867, y=0.04139466398744844
trange= ( 866445352.995 , 866446430.995 )                   
RA AVG: 273.6105398462901, DEC AVG: 18.00424756888499, ROLL AVG: 355.9914692945114
Loading raw6 file...                                        
7075686 events
Unpacking raw6 data...                                      
stim_coef0, stim_coef1 = -280492.08684388647, 0.00032960142264093945
writing table to /media/bekah/BekahA/background_test/pics_and_photonlists/e22118/e22118-fd-b00.parquet
                                                            
Runtime statistics:
 runtime		=	9.02 sec. = (0.15 min.)
  processed	   =   7022146 of 7075686 events.
rate		=	784257.89 photons/sec.

19.82 elapsed seconds, restarting timer
making images from /media/bekah/BekahA/background_test/pics_and_photonlists/e22118/e22118-fd-b00.parquet
indexing data and making WCS solution
making full-depth i

/home/bekah/gPhoton2/gPhoton/pipeline.py:231: UserWarning: lil=False is deprected and will be removed in a future release. Defaulting to lil=True.
  warnings.warn("lil=False is deprected and will be removed in a future release. Defaulting to lil=True.")


Processing eclipse 17019
Offsetting FUV image for eclipse 17019 at 30.681535821212 degrees.
Setting FUV offsets to x=-0.01435, y=-0.5525884348899571
trange= ( 836277276.995 , 836277391.995 )                   
RA AVG: 251.99460917074273, DEC AVG: 34.92358005133798, ROLL AVG: 298.9992377142042
Loading raw6 file...                                        
277310 events
Unpacking raw6 data...                                      
stim_coef0, stim_coef1 = 1022228.6156015451, -0.0012162701528634677
writing table to /media/bekah/BekahA/background_test/pics_and_photonlists/e17019/e17019-fd-b00.parquet
                                                            
Runtime statistics:
 runtime		=	1.37 sec. = (0.02 min.)
  processed	   =   258762 of 277310 events.
rate		=	201910.58 photons/sec.

3.8 elapsed seconds, restarting timer
making images from /media/bekah/BekahA/background_test/pics_and_photonlists/e17019/e17019-fd-b00.parquet
indexing data and making WCS solution
making full-depth image
0

KeyboardInterrupt: 

In [ ]:
# redo above without the .5 subtraction of flat field 